# Clone git repo

In [ ]:
!git clone https://{git_token}@github.com/{username}/{repository}

Cloning into 'ss23-drop-in-to-berlin'...
remote: Enumerating objects: 281, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 281 (delta 21), reused 25 (delta 21), pack-reused 238
Receiving objects: 100% (281/281), 6.51 MiB | 4.82 MiB/s, done.
Resolving deltas: 100% (143/143), done.


# Universal imports

In [1]:
from google.colab import drive
import pandas as pd

In [142]:
# Import BERT-ready data from Gdrive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# other git stuff

In [143]:
username = "TechLabs-Berlin"
repository = "ss23-drop-in-to-berlin"
git_token = ""

In [ ]:
cd /content/gdrive/MyDrive/TechLabs/{repository}/DL

In [ ]:
!git branch KNN_classifier recommender

In [22]:
!git checkout Keyword_classifier

DL/NN Classifier.ipynb: needs merge
error: you need to resolve your current index first


In [12]:
!git remote set-url origin https://{git_token}@github.com/{username}/{repository}

In [9]:
!git branch -r

  origin/HEAD -> origin/main
  origin/KNN_classifier
  origin/Keyword_classifier
  origin/Leo_local_pycharm
  origin/Recommender_API
  origin/about-page
  origin/backend
  origin/backend-fixes
  origin/details-page-styling
  origin/main
  origin/new-scraped-data
  origin/recommender
  origin/review-and-price-range-restcard
  origin/script-and-SearchResultsPage
  origin/star-rating-change
  origin/star-rating-fix


In [11]:
!git config --global user.email "phylanx@gmx.de"
!git config --global user.name "phylanxy"

In [141]:
!git add .

shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
fatal: Unable to read current working directory: Transport endpoint is not connected


In [16]:
!git commit -m "saving stuff from goolge colab in oder to switch branches"

[Recommender_API 44f1f3fd] saving stuff from goolge colab in oder to switch branches
 2 files changed, 2 insertions(+), 2 deletions(-)
 rewrite DL/NN Classifier.ipynb (75%)
 rewrite DL/keyword extraction.ipynb (77%)


In [21]:
!git pull

Auto-merging DL/NN Classifier.ipynb
CONFLICT (content): Merge conflict in DL/NN Classifier.ipynb
Automatic merge failed; fix conflicts and then commit the result.


# Installing requierd packages for keyword extraction


In [48]:
!pip install transformers
!pip install keybert
!pip install keyphrase-vectorizers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 66.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 97.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.2 MB/s eta 0:00:00
  Created wheel for keybert: filename=keybert-0.8.3-py3-none-any.whl size=39124 sha256=29ff05ff70144be368f097ea19d479f59873212802319cbcd239e3c06a2d219b
  Stored in directory: /root/.cache/pip/wheels/70/88/07/1a3bc11fd1dd5f89924a02dcbca89a3015e25e8faa31f904dc
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=b291ba0dcfe74d3cf9146b83b617cf2335e0f0d27f5e4940232443e7989af7b1
  

# Importing libraries for keyword extraction

In [49]:
from keybert import KeyBERT
#from keyphrase_vectorizers import KeyphraseCountVectorizer

# Loading the data & data cleaning


In [31]:
df = pd.read_csv("/content/gdrive/MyDrive/Ironhack/Final_project/df_with_model_input_no_NaNs.csv")

In [32]:
# function to remove items specified in a list
def preprocess_input(string, lst_to_remove):
  for item in lst_to_remove:
    string = string.replace(item, "")
  if len(string.split()) < 4:
    string = None
  else:
    pass
  return string

# create a list to remove strings that don't carry meaning
remove_lst = ["'review0': ","'review1': ","'review2': ","'review3': ","'review4': ", "'editorial_summary':", "berlin", "place", "restaurant", "restaurants", "eatery", "dishes", "meal", "meals", "dish", "food", "english"]

In [33]:
# drop old indeces
df.drop(["Unnamed: 0", "Unnamed: 0.1"], inplace=True, axis=1)

In [34]:
# turn model input into the review text without the items specified in remove_lst
df["model_input"] = df["model_input"].apply(lambda x: preprocess_input(str(x), remove_lst) if x != None else "")

In [35]:
# drop NaNs from model_input col, otherwise embedding won't work later on
df.dropna(subset="model_input", inplace=True)

In [53]:
# convert everything into str, add empty string for anything that's not a string (keyBERT won't accept NaNs as input, but I want to preserve the order of the inputs, for correct assignemetn of the outputs to the corresponding rows)
# and then create a list of texts for keyBERT to process
BERTs = list(df["model_input"])

In [37]:
import itertools as iter

In [38]:
def remove_attr(attr, text):
   return " ".join(list(iter.takewhile(lambda x: x != attr, text.split())))

In [40]:
df = df.applymap(lambda x: remove_attr("'types':", str(x)))

In [44]:
df.model_input[0]

'\'authentic german cuisine, friendly & fast served. huge portion, tasted like delicious home-cooked . highly recommended for salty lovers. however cash only, which is also quite german.\', \'very tasty german . worth a visit.\', \'great atmosphere with old s-bahn decorations. great space at the table and between tables. staff really nice and helpful, knowledgeable and advising for a good . good choice of and good size of , quality is good but could be better. price very reasonable, will go again. parking available inside.\', \'really amazing , finger licking and generous hospitality! really a hidden gem i never expected to find! their grill teller is really nice with 2 types of sauces. i loved the green pepper sauce in particular! it was soooo good! many families come here with kids and their kids menu is also good! the fish fingers with mash potatos and peas looked and tasted so good!\', "went here for a birthday and wasn\'t sure what to expect! service was great, was delicious, pric

# Build keyBERT model and extract the keywords

In [50]:
# Load a pre-trained BERT model for keyword extraction
kw_model = KeyBERT(model="all-MiniLM-L6-v2")

In [63]:
# Input document
document = BERTs

# initialize vectorizer with customized options for extracting zero or one adjective plus one or more nouns
#vectorizer = KeyphraseCountVectorizer(pos_pattern='<J.*>{0,1}<N.*>+', stop_words="english")

# define candidates
candidates = ["italian", "german", "indian", "japanese",  "vietnamnese", "turkish",
              "chinese", "greek", "mexican", "croatian", "thai", "russian", "persian", "iranian", "lebanese", "balkan","bavarian", "cafe",
              "nepalese", "korean", 'mediterranean', "fast food", "spanisch tapas", "tofu", "burger", "sushi", "grill", "fajitas", "shrimp", "tacos", "brewery",
              "doner kebab", "falafel", "currywurst", "steak", "fried chicken", "burrito", "pasta", "pizza", "lobster", "meat", "soup", "enchiladas", "schnitzel", "pork",
              "vegan", "vegetarian", "great services", "friendly staff", "authentic food", "bad service", "cozy", "wines", "dessert", "salads", "terrace"]

In [64]:
# Extract keywords using KeyBERT and pre-defined candidates
candidate_keywords = kw_model.extract_keywords(document, top_n=3, candidates=candidates)
seeded_keywords = kw_model.extract_keywords(document, top_n=3, seed_keywords=candidates)

In [94]:
# create dataframes for candidates_keywords and seeded_keywords
candidates_df = pd.DataFrame( tpls_to_lst(candidate_keywords), columns=["keywords_0", "keywords_1", "keywords_2"],)
seeded_df = pd.DataFrame(tpls_to_lst(seeded_keywords))

In [ ]:
# reset index of df to make indices match to candidate_df
df = df.reset_index().drop(["index", "Unnamed: 0.2"], axis=1)

In [125]:
# concat keywords to df
df = pd.concat([df, candidates_df], axis=1)

In [136]:
# create a new dataframe as a slice of the old one, using column names (new feature of pandas)
columns = ["reference", "keywords_0", "keywords_1", "keywords_2"]
df1 = pd.DataFrame(df, columns=columns)

In [140]:
df1

,reference,keywords_0,keywords_1,keywords_2
0,ChIJAergcnhJqEcRST9W0VduzeY,schnitzel,german,pork
1,ChIJaw12ev1QqEcRUApn-mBv2vM,wines,dessert,shrimp
2,ChIJWc98GMNRqEcRJKn-jkqXZgA,german,None,None
3,ChIJgTRA3fdNqEcRplfCByWlpO8,german,cozy,terrace
4,ChIJ80Ss-Z1PqEcRpsXmnT9JdUc,german,None,None
...,...,...,...,...
4268,ChIJqfomdtlQqEcRDbh3R4hqQiY,greek,None,None
4269,ChIJe_aa4NNQqEcRvGVumhyxim4,chinese,tofu,meat
4270,ChIJtdceUytXqEcRBvUYLcJKo2M,indian,None,None
4271,ChIJr0qHXtZQqEcRI8A-t0OCzg8,italian,pasta,pizza


In [139]:
df1.to_json("/content/gdrive/MyDrive/TechLabs/ss23-drop-in-to-berlin/DL/candidate_keywords.json", orient="records")

In [74]:
# function that returns only keywords without probabilities
def tpls_to_lst(tpls):
  keywords_lst = []
  for sublist in tpls:
    word_lst = [word[0] for word in sublist]
    keywords_lst.append(word_lst)
  return keywords_lst

In [ ]:
# extract words from touples
keywords_lst = tpls_to_lst(keywords)
keywords_lst

In [ ]:
# build a dataframe from the keywords and store it in gdrive as a csv file
keywords_df = pd.DataFrame(keywords_lst)

In [ ]:
# save to keywords.csv if needed
keywords_df.to_csv("/content/gdrive/MyDrive/Techlabs/keywords.csv", index_label=None)

In [ ]:
# try keyword extraction for possible user input
user_input = "I want a restaurant that is beautiful and where the waiters are very friendly, the desserts are delicious and I can bring my dog.".replace("restaurant", "")

# Extract keywords using KeyBERT
user_keywords = kw_model.extract_keywords(user_input, vectorizer=vectorizer, top_n=6)

user_keywords

In [ ]:
keywords_lst

# Clustering keywords into topics




## load data


In [ ]:
# load keywords from csv
keywords_df = pd.read_csv("/content/gdrive/MyDrive/Techlabs/keywords.csv")

FileNotFoundError: ignored

In [ ]:
# get all unique keywords and save in keywords_lst
keywords_lst = list(set([item for sublist in keywords_df.applymap(str).values.tolist() for item in sublist]))
keywords_lst

## build BERTopic model on top of keyBERTs

In [ ]:
!pip install --upgrade numpy numba

In [ ]:
!pip install bertopic

In [ ]:
from bertopic import BERTopic
from bertopic.representation import MaximalMarginalRelevance
from sentence_transformers import SentenceTransformer
from transformers import pipeline
from bertopic.representation import KeyBERTInspired
from hdbscan import HDBSCAN

In [ ]:
#0 create embeddings by using a pipeline as a high-level helper
embedding_model = pipeline("feature-extraction", model="vocab-transformers/distilbert-word2vec_256k-MLM_best")

#1 customize clustering model
hdbscan_model = HDBSCAN(min_cluster_size=10, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

#2 use keyBERT to define topic labels
representation_model = KeyBERTInspired()

#3 built topic model and fit on the keywords
topic_model = BERTopic(embedding_model=embedding_model, hdbscan_model=hdbscan_model)

In [ ]:
#4 fit & transform
docs = keywords_lst
topics, probs = topic_model.fit_transform(docs)

In [ ]:
from joblib import dump, load

# save model in a joblib file (safer than pickle)
dump(topic_model, '/content/gdrive/MyDrive/Techlabs/keyBERT_model_word2vec_eom_GPU.joblib')

['/content/gdrive/MyDrive/Techlabs/keyBERT_model_word2vec_eom_GPU.joblib']

### visualize resulting topics

In [ ]:
from sklearn.datasets import fetch_20newsgroups
from bertopic import BERTopic
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
# create wordcloud for each topic
def create_wordcloud(topic_model, topic):
    text = {word: value for word, value in topic_model.get_topic(topic)}
    wc = WordCloud(background_color="white", max_words=1000)
    wc.generate_from_frequencies(text)
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")
    plt.show()

# Show word cloud
for topic in topic_model.get_topic_info()["Topic"]:
  create_wordcloud(topic_model, topic=topic)

In [ ]:
candidates = ["vegetarian", "sushi", "ramen", "dessert", ]

In [ ]:
topic_model.get_topic(1)

In [ ]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,2843,-1_pizza_best_pizzas_authentic,"[pizza, best, pizzas, authentic, style, food, ...","[best chicken burger, good quality pizzas, coz..."
1,0,250,0_spandau_potsdam_gate_platz,"[spandau, potsdam, gate, platz, rathaus, kaise...","[brandenburger gate, rathaus spandau, brandenb..."
2,1,169,1_sushi_sushis_nigiri_tempura,"[sushi, sushis, nigiri, tempura, rolls, top, p...","[good sushi, best sushi, sushi]"
3,2,161,2_schwarma_makali_lietzensee_nem,"[schwarma, makali, lietzensee, nem, kibbeh, ku...","[great schwarma, schwarma, tasty chicken schwa..."
4,3,147,3_burger_burgers_cheese_king,"[burger, burgers, cheese, king, cheeseburger, ...","[delicious burger, burger, chicken burger]"
...,...,...,...,...,...
177,176,10,176_wines_wine_serbian_macedonian,"[wines, wine, serbian, macedonian, croatian, i...","[german wines, greek wines, italian wines]"
178,177,10,177_station_bahn_train_gesunbrunen,"[station, bahn, train, gesunbrunen, weg, sudkr...","[ostkreuz train station, picturesque nikolasse..."
179,178,10,178_thuringian_blau_schwabischen_schiller,"[thuringian, blau, schwabischen, schiller, obe...","[kaffee crema, thuringian bread, thuringian br..."
180,179,10,179_training_employees_trainees_operation,"[training, employees, trainees, operation, job...","[other employees, training operation, training]"


In [ ]:
topic_model.visualize_topics()

In [ ]:
topic_model.visualize_hierarchy()

## custom approach

In [ ]:
from transformers import BertTokenizer, BertModel
import torch

In [ ]:
# Load a pre-trained BERT model and tokenizer
model_name = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# Prepare your input data
input_text = keywords_lst[:2]

# Tokenize and convert to BERT input format
tokenized_input = tokenizer(*input_text, padding=True, truncation=True, return_tensors="pt")

# Generate BERT embeddings
with torch.no_grad():
    outputs = model(**tokenized_input)
    embeddings2 = outputs.last_hidden_state

In [ ]:
embeddings.size(), embeddings2.size()

(torch.Size([6, 6, 384]), torch.Size([6, 12, 384]))

### getting cosine similarity between two vectors

In [ ]:
import torch
import torch.nn.functional as F

In [ ]:
# Define two vectors as PyTorch tensors
vector1 = embeddings2[:,:6,:]
vector2 = embeddings2[:,6:12,:]

# Calculate cosine similarity
similarity = F.cosine_similarity(vector1, vector2, dim=0)

# Print the cosine similarity
print("Cosine Similarity:", similarity)


Cosine Similarity: tensor([[ 0.5551,  0.3613,  0.7036,  ...,  0.3346,  0.4634,  0.3585],
        [ 0.6444,  0.2337, -0.2471,  ...,  0.4694,  0.6556,  0.2050],
        [ 0.7086,  0.4504, -0.1781,  ...,  0.5310,  0.2568,  0.1821],
        [ 0.5031,  0.7434,  0.0050,  ...,  0.5892,  0.6540,  0.2148],
        [ 0.1147,  0.7820, -0.4728,  ..., -0.2439,  0.6435,  0.1375],
        [ 0.9084,  0.8012,  0.2966,  ...,  0.0740,  0.7396,  0.5129]])


In [ ]:
similarity.size(), vector1.size(), vector2.size()

(torch.Size([6, 384]), torch.Size([6, 6, 384]), torch.Size([6, 6, 384]))

## approach this with fastai

In [ ]:
!pip install fastai

In [ ]:
import fastai
from fastai.text.all import *

### initialize tokenizer

In [ ]:
# this seems to be able to tokenize my input
spacy = WordTokenizer()

# this somehow doesn't work...
tkn = Tokenizer(spacy)

In [ ]:
# creating tokens from input
txt = BERTs
toks = spacy(txt)

In [ ]:
# setup the numericalizer
# NOT working atm, need to fix
num = Numericalize()
num.setup(toks)
coll_repr(num.vocab,100)

'(#7720) [\'xxunk\',\'xxpad\',\'xxbos\',\'xxeos\',\'xxfld\',\'xxrep\',\'xxwrep\',\'xxup\',\'xxmaj\',\'.\',\',\',\'the\',"\'",\'and\',\'a\',\'was\',\'i\',\'"\',\'to\',\'is\',\'food\',\'!\',\'it\',\'in\',\'of\',\'very\',\'for\',\' \',\'good\',\'with\',\'we\',\'you\',\'but\',\'are\',\'not\',\'service\',\'nice\',\'great\',\'this\',\'that\',\'friendly\',\'they\',\'were\',"n\'t",\'have\',\'there\',\'delicious\',\'-\',\'had\',\'staff\',\'really\',\'on\',\'so\',\'my\',"\'s",\'as\',\'also\',\'at\',\'be\',\'here\',\'tasty\',\')\',\'(\',\'all\',\'can\',\'like\',\'best\',\'one\',\'from\',\'if\',\'time\',\'pizza\',\'recommend\',\'just\',\':\',\'would\',\'do\',\'only\',\'will\',\'super\',\'prices\',\'price\',\'well\',\'too\',\'dishes\',\'ordered\',\'an\',\'fresh\',\'amazing\',\'back\',\'always\',\'menu\',\'more\',\'again\',\'or\',\'taste\',\'which\',\'definitely\',\'quality\',\'me\'...]'

In [ ]:
num(txt[0])

TensorText([  12,   14,  874,  ..., 3698,   12,  155])

In [ ]:
txt[0]

'german cuisine german food authentic german schnitzel cuisine great food'